#Run this before starting
!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"


In [63]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax
import torch.optim as optim
from torch_geometric.nn.conv import feast_conv


In [64]:
def normal(tensor, mean, std):
    if tensor is not None:
        tensor.data.normal_(mean, std)

class FeaStConv(MessagePassing):
    def __init__(self,
                 in_channels,
                 out_channels,
                 heads=8,
                 bias=True,
                 t_inv=True,
                 **kwargs):
        super(FeaStConv, self).__init__(aggr='mean', **kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.heads = heads
        self.t_inv = t_inv

        self.weight = Parameter(torch.Tensor(in_channels,
                                             heads * out_channels))
        self.u = Parameter(torch.Tensor(in_channels, heads))
        self.c = Parameter(torch.Tensor(heads))
        if not self.t_inv:
            self.v = Parameter(torch.Tensor(in_channels, heads))

        if bias:
            self.bias = Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

    def reset_parameters(self):
        normal(self.weight, mean=0, std=0.1)
        normal(self.u, mean=0, std=0.1)
        normal(self.c, mean=0, std=0.1)
        normal(self.bias, mean=0, std=0.1)
        if not self.t_inv:
            normal(self.v, mean=0, std=0.1)

    def forward(self, x, edge_index):
        edge_index, _ = remove_self_loops(edge_index)
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        return self.propagate(edge_index, x=x, num_nodes=x.size(0))

    def message(self, x_i, x_j):
        # dim: x_i, [E, F_in];
        if self.t_inv:
            # with translation invariance
            q = torch.mm((x_i - x_j), self.u) + self.c  #[E, heads]
        else:
            q = torch.mm(x_i, self.u) + torch.mm(x_j, self.v) + self.c
        q = F.softmax(q, dim=1)  #[E, heads]

        x_j = torch.mm(x_j, self.weight).view(-1, self.heads,
                                              self.out_channels)
        return (x_j * q.view(-1, self.heads, 1)).sum(dim=1)

    def update(self, aggr_out):
        if self.bias is not None:
            aggr_out = aggr_out + self.bias
        return aggr_out

    def __repr__(self):
        return '{}({}, {}, heads={})'.format(self.__class__.__name__,
                                             self.in_channels,
                                             self.out_channels, self.heads)


In [65]:
class FeaStNet(torch.nn.Module):
    def __init__(self, in_channels, num_classes, heads, t_inv=True):
        super(FeaStNet, self).__init__()

        self.fc0 = nn.Linear(in_channels, 16)
        self.conv1 = feast_conv(16, 32, heads=heads, t_inv=t_inv)
        self.conv2 = feast_conv(32, 64, heads=heads, t_inv=t_inv)
        self.conv3 = feast_conv(64, 128, heads=heads, t_inv=t_inv)
        self.fc1 = nn.Linear(128, 256)
        self.fc2 = nn.Linear(256, num_classes)

        self.reset_parameters()

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()
        self.conv3.reset_parameters()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.elu(self.fc0(x))
        x = F.elu(self.conv1(x, edge_index))
        x = F.elu(self.conv2(x, edge_index))
        x = F.elu(self.conv3(x, edge_index))
        x = F.elu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


In [124]:
#data
#TxtFile to lists
import numpy as np 

#data = np.loadtxt('C:\\Users\\eliasak\\OneDrive - NTNU\\Master thesis\\07_ML\\ArchGNN\\data.txt')

values_list = []  # create an empty list to store the values for each row
x = []
z = []
m = []
l = []

with open("C:\\Users\kt\\OneDrive - NTNU\\Master thesis\\03_Herman\\mega-data.txt", "r") as f:
    for line in f:  # loop over each line in the file
        values = line.strip().split("\t")  # split the line into a list of strings

        xL = []
        zL = []
        mL = []
        lL = []

        for i in range(20):
            xL.append(round(float(values[i]),3))
            zL.append(round(float(values[i+20]),3))
            mL.append(round(float(values[i+20*2]),3))
            try:
                lL.append(round(float(values[i+20*3]),3))
            except:
                None
        x.append(xL)
        z.append(zL)
        m.append(mL)
        l.append(lL)


dataY = []

dataX = []
for i in range(len(x)):
    dataXs = []
    for j in range(0,len(x[i])):
        dataX0 = []
        dataX0.append(j)
        dataX0.append(x[i][j])
        dataX0.append(z[i][j])
        dataXs.append(dataX0)
    dataX.append(dataXs)

dataX = np.array(dataX)
dataY = np.array(m)

dataEgdeIndex = []
for i in range(len(x[0])-1):
    dataEgdeIndex.append([i,i+1])
    dataEgdeIndex.append([i+1,i])  

dataEgdeIndex = np.array(dataEgdeIndex) 
dataEgdeIndex =np.transpose(dataEgdeIndex)


input_data = dataX
target_data = dataY
edge_index = dataEgdeIndex
dataX = torch.from_numpy(dataX)
dataY = torch.from_numpy(dataY)
edge_index = torch.from_numpy(edge_index)
edge_index = edge_index.to(torch.long)
dataX = dataX.to(torch.float)
dataY = dataY.to(torch.float)


dataset = []
for i in range(dataX.shape[0]):
    dataset.append(Data(x=dataX[i], edge_index=edge_index, y=dataY[i]))

train_loader = dataset[:200]
test_loader = dataset[200:]
print(dataX.dtype)
print(dataY.dtype)


# one data object from train_loader:
graph = train_loader[0]

print(graph.x.shape)
print(graph.edge_index.shape)
print(graph.y.shape)

torch.float32
torch.float32
torch.Size([20, 3])
torch.Size([2, 38])
torch.Size([20])


In [125]:
class ArchNN(torch.nn.Module):
    def __init__(self, in_channels, num_classes, heads, t_inv = True):
        super(ArchNN, self).__init__()
        self.fc0 = nn.Linear(in_channels, 16)
        self.conv1 = FeaStConv(16, 32, heads=heads, t_inv=t_inv)
        self.conv2 = FeaStConv(32, 64, heads=heads, t_inv=t_inv)
        self.conv3 = FeaStConv(64, 128, heads=heads, t_inv=t_inv)
        self.fc1 = nn.Linear(128, 256)
        self.fc2 = nn.Linear(256, 1)

        self.reset_parameters()

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()
        self.conv3.reset_parameters()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.elu(self.fc0(x))
        x = F.elu(self.conv1(x, edge_index))
        x = F.elu(self.conv2(x, edge_index))
        x = F.elu(self.conv3(x, edge_index))
        x = F.elu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        #F.log_softmax(x, dijm=1)
        x = torch.squeeze(x, dim=1)
        return x




In [126]:
import time
import torch
import torch.nn.functional as F


def print_info(info):
    message = ('Epoch: {}/{}, Duration: {:.3f}s,'
               'Train Loss: {:.4f}, Test Loss:{:.4f}').format(
                   info['current_epoch'], info['epochs'], info['t_duration'],
                   info['train_loss'], info['test_loss'])
    print(message)


def run(model, train_loader, test_loader, num_nodes, epochs, optimizer):

    for epoch in range(1, epochs + 1):
        t = time.time()
        train_loss = train(model, train_loader, optimizer)
        t_duration = time.time() - t
        test_loss = test(model, test_loader, num_nodes)
        eval_info = {
            'train_loss': train_loss,
            'test_loss': test_loss,
            'current_epoch': epoch,
            'epochs': epochs,
            't_duration': t_duration
        }

        print_info(eval_info)


def train(model, train_loader, optimizer):
    model.train()

    total_loss = 0
    for idx, data in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        #log_probs = F.log_softmax(output, dim=1).double()
        #print(log_probs.dtype)
        #print(data.y.dtype)
        loss = F.mse_loss(output, data.y)
        #loss = F.nll_loss(log_probs, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)


def test(model, test_loader, num_nodes):
    model.eval()
    correct = 0
    total_loss = 0
    n_graphs = 0
    with torch.no_grad():
        for idx, data in enumerate(test_loader):
            out = model(data)
            total_loss += F.mse_loss(out, data.y).item()
            #pred = out.max(1)[1]
            #correct += pred.eq(data.y).sum().item()
            #n_graphs += data.num_graphs
    return total_loss / len(test_loader)


In [127]:
#runner
num_nodes = train_loader[0].x.shape[0]
num_features = train_loader[0].x.shape[1]

model = ArchNN(num_features, num_nodes, heads=10)

optimizer = optim.Adam(model.parameters(),
                       lr=0.001)


run(model, train_loader, test_loader, num_nodes, 10, optimizer)

Epoch: 1/10, Duration: 1.599s,Train Loss: 31.1131, Test Loss:455.2588
Epoch: 2/10, Duration: 1.546s,Train Loss: 3.3077, Test Loss:464.3560
Epoch: 3/10, Duration: 1.584s,Train Loss: 4.7777, Test Loss:384.6578
Epoch: 4/10, Duration: 1.753s,Train Loss: 4.9031, Test Loss:308.6826
Epoch: 5/10, Duration: 1.698s,Train Loss: 4.9050, Test Loss:358.8964
Epoch: 6/10, Duration: 1.771s,Train Loss: 3.6502, Test Loss:347.6817
Epoch: 7/10, Duration: 1.984s,Train Loss: 3.9746, Test Loss:370.7009
Epoch: 8/10, Duration: 1.649s,Train Loss: 2.8581, Test Loss:361.9896
Epoch: 9/10, Duration: 1.912s,Train Loss: 3.2240, Test Loss:353.9258
Epoch: 10/10, Duration: 1.737s,Train Loss: 2.7431, Test Loss:359.0310


In [38]:
#OLD
# Define a simple GNN model
class MyGNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MyGNN, self).__init__()
        
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.conv3(x, edge_index)
        return x

# Define a simple training function for the GNN model
def train(model, data, optimizer):
    model.train()
    optimizer.zero_grad()
    output = model(data.x, data.edge_index)
    loss = F.mse_loss(output, data.y)
    loss.backward()
    optimizer.step()
    return loss.item()

# Define a simple testing function for the GNN model
def test(model, data):
    model.eval()
    output = model(data.x, data.edge_index)
    return output

# Set up the input data
x = torch.tensor([
    [0,0,1,1,1,0,0,0],
    [10,5,0,0,0,0,5,0],
    [20,10,0,0,0,0,5,0],
    [30,5,0,0,0,0,5,0],
    [40,0,1,1,1,0,0,0]
], dtype=torch.float)

edge_index = torch.tensor([
    [0,1], [1,0], [1,2], [2,1], [2,3], [3,2], [3,4], [4,3]
], dtype=torch.long).t()

# Set up the output data (nodal forces and moments)
y = torch.tensor([
    [0,0,0,0,0,10,20,0],
    [0,0,0,0,0,5,0,0],
    [0,0,0,0,0,5,0,0],
    [0,0,0,0,0,5,0,0],
    [0,0,0,0,0,10,20,0]
], dtype=torch.float)

# Create a Data object that encapsulates the input and output data
data = Data(x=x, edge_index=edge_index, y=y)

# Initialize the GNN model
input_dim = x.shape[1]
output_dim = y.shape[1]
hidden_dim = 16
model = MyGNN(input_dim, hidden_dim, output_dim)

# Define the optimizer and the number of epochs for training
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
num_epochs = 100

# Train the GNN model
for epoch in range(num_epochs):
    loss = train(model, data, optimizer)
    print('Epoch {}: loss={}'.format(epoch, loss))

# Test the GNN model on a new input example
x_new = torch.tensor([
    [0,0,1,1,1,0,0,0],
    [10,5,0,0,0,0,5,0],
    [20,10,0,0,0,0,5,0],
    [30,5,0,0,0,0,5,0]
])
print(edge_index)

Epoch 0: loss=27.194686889648438
Epoch 1: loss=23.968759536743164
Epoch 2: loss=21.4787540435791
Epoch 3: loss=19.75748062133789
Epoch 4: loss=18.83110237121582
Epoch 5: loss=18.46010971069336
Epoch 6: loss=18.401479721069336
Epoch 7: loss=18.3200740814209
Epoch 8: loss=18.0425968170166
Epoch 9: loss=17.594493865966797
Epoch 10: loss=17.073143005371094
Epoch 11: loss=16.612995147705078
Epoch 12: loss=16.298181533813477
Epoch 13: loss=16.103126525878906
Epoch 14: loss=16.005006790161133
Epoch 15: loss=15.95953369140625
Epoch 16: loss=15.92503547668457
Epoch 17: loss=15.877996444702148
Epoch 18: loss=15.810765266418457
Epoch 19: loss=15.72706127166748
Epoch 20: loss=15.643503189086914
Epoch 21: loss=15.57574462890625
Epoch 22: loss=15.531913757324219
Epoch 23: loss=15.508995056152344
Epoch 24: loss=15.494709968566895
Epoch 25: loss=15.474119186401367
Epoch 26: loss=15.437162399291992
Epoch 27: loss=15.396413803100586
Epoch 28: loss=15.341238021850586
Epoch 29: loss=15.272732734680176
Epo